# Some preparation steps

1. Install `miniforge3` on your laptop in order to use Python and other packages.
2. Install python packages using `pip` as below.

In [ ]:
# !curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
# !bash Miniforge3-$(uname)-$(uname -m).sh

In [ ]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib beautifulsoup4 pyperclip openai scikit-learn

# Importing necessary functions

1. Make sure to import functions to properly display markdowns.
2. Import necessary functions we implemented from `retrieve_news_alert.py` and `summarize_with_openai`.

In [1]:
from IPython.display import Markdown, display

import numpy as np

from retrieve_news_alert import *
from summarize_with_openai import *

# Prepare API access

1. Prepare GMail authorization: if already done, it should happen without any prompt.
2. Read OpenAI API key from `openai_api_key.txt`.

In [2]:
service = authenticate_gmail()
with open('openai_api_key.txt', 'r') as f:
    key = f.readline().strip()

# Retrieve initial emails

1. Retrieve all emails from Bloomberg with "Alert" in the subject from Gmail.
2. Extract the main contents from HTML.
3. Create a topic list using OpenAI's API.

In [3]:
emails = get_emails(service, days=1, query=f'from:Bloomberg subject:Alert')
articles = format_emails(emails)

Total 47 news articles have been retrieved.


In [4]:
topics = extract_topics_from_titles(key, articles, instructions=[
                                   "Please sort them according to their prominence and cluster them as closely as possible.",
                                   "Please be as concise as possible.", 
                                   "Please provie only the cluster titles not the actual news titles.",
                                   "Please keep the bullet point list in one level.",
                                   "Please use markdown."
                               ])

display(Markdown(topics))

- **US-China Relations:**
  - Chinese ‘Provocations’ in Military Drills Around Taiwan
  - Taiwan’s Status Risks Igniting a US-China Clash
  - US Extends Some China Tariffs Exclusions 
- **Political Developments:**
  - Rapid Deployment to Haiti After Missionaries Killed
  - Biden Extends Tariff Exemptions from China
  - Kennedy Hits Trump’s Covid Response
  - Biden’s Labor Board Picks and Trump Winning Election Clash
  - Senator Urges Changes to Proposed Hydrogen Tax Credit Rules
- **Economic Impact:**
  - $5 Trillion Risk from China ‘Blockade Simulation’
  - Power Utilities Ask Court to Block Biden Pollution Curbs
  - Louisiana Criminalizes Possessing Abortion Pills
  - US Debt Path Seen Entering a Danger Zone
- **International Relations:**
  - Hong Kong, Guangdong, Macau Officials Visit US
  - US to Send Team to Cairo for Rafah Crossing Talks
  - UAE President to Visit China After Invite From Xi
- **Tech and Trade Policies:**
  - Schwarzman Supporting Trump
  - FDIC Delayed Misconduct Allegations Report
  - Biden's Tech Diplomacy with Kenya
- **Legislation and Regulations:**
  - FDIC Candidate for New Chair
  - House Appropriators Begin FY 2025 Funding Sprint
  - Federal Privacy Bill Confusion Over Using Kids’ Data
- **Environmental Concerns:**
  - Climate Talks Visit by HK, Guangdong, Macau Officials
- **Labor and Union Actions:**
  - Unions See Opportunities in Chip Factories
  - Chip Factories are Unions’ Next Target for Biden

# Retrieve topic-relevant emails

1. Input the topic string based on the topic list above.
2. Retreive the emails with the new query from GMail.
3. Extract the main content from HTML.

In [5]:
topic = input("Please write the topic of your interest")
emails = get_emails(service, days=7, query=f'from:Bloomberg {topic}')
articles = format_emails(emails)

Total 8 news articles have been retrieved.


# Summarize individual articles

1. Summarize each article using OpenAI's GPT.
2. Print the summarized articles.

In [6]:
article_summaries = [
    {'title': article['title'], 'content': summarize_articles(key, [article], 
                                                              n_paragraphs=3, 
                                                              instructions=[
                                                                  "If possible, use bullet points to make it easily readable.",
                                                                  "Do not hesitate to paraphrase in order to make it readable and informative.",
                                                                  "Please use markdown."])} 
    for article in articles
]

summary_text = ""

for i, article in enumerate(article_summaries):
    summary_text += f"### {i+1} {article['title']}\n\n {article['content']}\n\n"

display(Markdown(
    f"## Selected Articles\n\n{summary_text}"
))

## Selected Articles

### 1 Unions See Opportunities in Chip Factories: Washington Edition

 ### Summary
**Unions in the semiconductor industry:**
- Unions are looking to organize workers in the semiconductor industry amidst President Biden's economic policies.
- Companies like Taiwan Semiconductor Manufacturing Co. and Intel Corp. are set to receive funding under the 2022 Chips Act, creating opportunities for labor unions.
- The Communications Workers of America (CWA) is focusing on organizing chip-making workers, similar to the United Auto Workers' drive in the auto industry.

**Challenges and Opportunities:**
- Organizing the chip industry presents unique challenges due to a lack of unionized semiconductor workers in the US.
- Intel, a major player in the industry, has shown reluctance towards union talks, posing a hurdle for the CWA.
- Despite challenges, the CWA has secured labor peace negotiations with Micron, potentially paving the way for successful future campaigns.

**Future Perspectives and Developments:**
- As the Biden administration pushes for a national-security-driven manufacturing boom, there is an opportunity to leverage the Chips Act to promote organized labor.
- The administration's negotiations with companies capable of advanced chip manufacturing could influence industry practices and labor relations.
- The article also highlights upcoming political events, including speeches by Biden and Trump, as well as the G7 summit and 2024 presidential debates.

### 2 Biden’s Labor Board Picks Set Up Clash If Trump Wins Election

 - President Joe Biden's nominations to the National Labor Relations Board (NLRB) could ensure Democratic control of the board majority until 2026, even if Republican Donald Trump wins the election.
- Biden's renomination of NLRB Chair Lauren McFerran and his selection of attorney Joshua Ditelberg for the open Republican seat has drawn criticism from business advocates, who view it as an attempt to solidify radical policies.
- The confirmation of McFerran and Ditelberg would secure the NLRB's composition until 2026, potentially impacting significant labor rulings and setting up a clash between Senate Democrats, organized labor, Senate Republicans, and the business lobby.

- The possibility of Democrats maintaining control of the NLRB majority under a Republican administration is concerning to the management community, as it could lead to unfavorable outcomes for employers.
- Senate consideration of McFerran's renomination is seen as potentially influencing the election process in November, with some arguing that Ditelberg's nomination should be treated separately.
- Biden's departure from the bipartisan tradition of pairing NLRB nominations raises questions about the nomination process, particularly regarding the timing and political motivations involved.

- Despite the delayed nomination of Ditelberg and the challenges associated with Senate approval, the current Democratic majority in the Senate suggests that his confirmation alongside McFerran is expected.
- The ongoing political dynamics surrounding the NLRB appointments highlight the importance of balancing interests between labor and business stakeholders, with potential implications for future labor policy decisions and legal precedents.
- The confirmation process for McFerran and Ditelberg, and the potential implications of Democratic control of the NLRB, are subjects of intense scrutiny and debate within the political and legal spheres.

### 3 Boeing Working With Union, FAA to Review Employee Concerns (1)

 **Summary:**

- Boeing is collaborating with its largest labor union and the FAA to review safety concerns raised by employees through its Speak Up program, aiming to address issues regarding the company's safety culture. 
- The efforts to encourage more employees to voice their concerns and ensure they are heard have intensified following a recent incident involving a door plug on a 737 Max aircraft. 
- The safety report highlights actions taken over the past year and precedes Boeing's upcoming delivery of a 90-day plan to the FAA outlining corrective actions for quality control issues at its factories. The company has seen a significant increase in Speak Up submissions and has conducted product safety training for a large number of employees, while also implementing measures to improve production processes.

*Source:* [Bloomberg: Boeing Working With Union, FAA to Review Employee Concerns](insert link)

### 4 Chip Factories are Unions’ Next Target in Test For Biden

 ### News Summary:

- **Unions Targeting Chip Factories for Biden's Manufacturing Boom Promise:**
  - Unions like the Communications Workers of America are focusing on semiconductor companies to test President Biden's pledge for a manufacturing revival.
  - The CWA is in talks with Micron Technology Inc. for a labor peace agreement covering a $50 billion investment in New York, under which the union would not disrupt operations.
  - Biden's economic policies offer subsidies for tech sectors like semiconductors, with the aim of creating jobs and fostering organized labor.

- **Role of Extreme Ultraviolet Lithography Machines in Semiconductor Production:**
  - Extreme ultraviolet lithography machines are crucial for semiconductor manufacturing that drives the artificial intelligence revolution and global economy growth.
  - ASML in the Netherlands is a key player in producing these high-tech machines, with restrictions on sales to China enforced by the US government.
  - Organized labor campaigns are extending to new factories in line with Biden's policies, potentially challenging industry leaders' historical resistance to unions.

- **Leveraging Biden's Chips Act for Labor Rights in Semiconductor Industry:**
  - Labor unions seek stronger commitments from semiconductor giants like Intel, Micron, and TSMC, potentially using political influence to negotiate neutrality agreements.
  - Chips Act grants are seen as a tool to enforce labor protections, with funding distribution contingent on companies meeting benchmarks and supporting union efforts.
  - While specifics of labor agreements under the Chips Act are undisclosed, the Biden administration aims to facilitate discussions between awardees and industrial unions to address workforce issues.

### 5 After the Election, a Big Tax Fight Looms: Washington Edition

 ### Summary:
- A looming battle over taxes in 2025 following the 2024 election is expected to directly impact Americans' wallets as Trump-era individual tax cuts are set to expire.
- Both Trump and Biden have outlined their tax policy stances, with Trump proposing tax cuts for businesses and individuals, while Biden aims to raise taxes on corporations and maintain certain individual tax cuts.
- The potential renewal of individual tax rates could cost upwards of $4 trillion over a decade, with concerns raised about the impact on federal debt and future generations.

### Key Points:
- A survey shows that 77% of swing state voters support a billionaires tax to save Social Security.
- The Biden administration is looking to allocate funding to Kenya under the Chips and Science Act to bolster tech industry relationships.
- New home purchases in the US have declined due to affordability challenges caused by high prices and mortgage rates.

### Upcoming Events:
- Biden will host Kenyan President Ruto for a state dinner at the White House.
- The University of Michigan's final read of consumer sentiment for May will be released tomorrow.
- The Republican and Democratic conventions are scheduled for July and August, respectively.
- The first general election US presidential debate of 2024 will be hosted by CNN on June 27.
- The Group of Seven leaders summit is set to convene in Italy from June 13-15.

### 6 Biden Moves to Extend Labor Board Chief as He Courts Unions

 - President Joe Biden is nominating Lauren McFerran for a new five-year term as Chair of the US National Labor Relations Board (NLRB) to secure a Democrat majority on the board. McFerran has been instrumental in issuing union-friendly rulings during her tenure. Biden is also nominating labor lawyer Joshua Ditelberg for a Republican seat on the board which would ensure terms up to 2029 and 2027 for McFerran and Ditelberg, respectively.
- The NLRB has faced challenges to its constitutionality by companies like SpaceX, Amazon, and Starbucks. Biden, positioning himself as pro-union, is relying on labor support for his re-election bid against Trump, who is also vying for working-class votes. The union vote in battleground states like Pennsylvania, Michigan, Wisconsin, Nevada, and Georgia could play a crucial role in the upcoming election.
- Under Biden's administration, the NLRB has made it easier for workers to organize and win collective bargaining if their employer violates the law. Trump's return to the White House could reshape federal policies on unionizing, as his conservative allies oppose the union drives encouraged by Biden. The upcoming NLRB appointments and potential election outcomes have significant implications for labor policies in the US.

### 7 College Athletes Barred From Employment Status in New GOP Bill

 ### College Athletes Employment Status Bill

- **Republican Bill Overview**: 
  - A new GOP bill aims to prevent college athletes from being classified as employees.
  - The Protecting Student Athletes’ Economic Freedom Act would establish that athletes cannot be considered employees under any regulation or law.
  - The bill is presented as a way to maintain the balance between academics and athletics in college sports.

- **Political Debate and Court Cases**:
  - Capitol Hill is embroiled in a debate over college athlete employment rights following the Dartmouth basketball team's move to unionize.
  - Republicans view athlete employment classification as a threat to college sports, while Democrats are weighing the consequences of such classification.
  - The bill would exempt athletes from minimum wage requirements and labor union rights laws.

- **Additional Considerations**:
  - Issues such as workplace safety and protection from employment discrimination would be exempted for college athletes under this bill.
  - Lawmakers are also discussing potential regulations regarding students' ability to profit from their name, image, and likeness.
  - Noteworthy court cases involving the NCAA and student-athletes are also ongoing, showcasing the complexity of this issue.

This legislation represents a pivotal moment in the ongoing debate surrounding the employment status of college athletes, impacting not only their rights but also the future landscape of collegiate sports.

### 8 American Air, Gate Gourmet Pressed on Contracts to Avoid Strikes

 - **Negotiations Pressured to Avoid Strikes:**
  - President Biden's administration is urging American Airlines and Gate Gourmet to reach new contracts for flight attendants and airline caterers to prevent labor strikes.
  - The National Mediation Board has set an end-of-May deadline for both parties to reach an agreement.
  - If no deal is reached, a 30-day "cooling off" period would ensue, potentially leading to strikes that could disrupt air travel.

- **Political and Economic Implications:**
  - The labor talks present a challenge for Biden's reelection hopes as they could impact support from organized labor.
  - The pressure from Congress and the White House aims to expedite negotiations for long-overdue contracts.
  - American Airlines has proposed a 17% pay increase, falling short of the 28% sought by the flight attendants' union.

- **Historical Context and Presidential Influence:**
  - Previous strikes in the airline industry, including at American Airlines in 1993, highlight the impact of presidential intervention.
  - Biden's support for labor unions contrasts with challenges in gaining support from rank-and-file workers.
  - Southwest Airlines' recent agreement sets a benchmark for future negotiations at American and United Airlines.



# Summarize all articles based on their respective summaries

1. Add the article numbers from above into `exclude_articles`, e.g., `exclude_articles=[1, 2]`, to exclude these irrelevant articles.
2. Use OpenAI API to summarize these articles.

In [12]:
exclude_articles = [5, 7, 8]
included_articles = [i for i in range(len(articles)) if i+1 not in exclude_articles]

included_article_summaries = []

for i in included_articles:
    included_article_summaries.append(article_summaries[i])

display("Included articles: " + ", ".join([str(i+1) for i in included_articles]))

'Included articles: 1, 2, 3, 4, 6'

In [15]:
summary = summarize_articles(key, 
                             included_article_summaries, 
                             n_paragraphs=5, 
                             instructions=[
                                 "If possible, use bullet points to make it easily readable.",
                                 "Please avoid making any subjective assessment.", 
                                 "Please draw insights across multiple articles.",
                                 "Do not hesitate to paraphrase in order to make it readable and informative.",
                                 "Please use markdown."])

display(Markdown(summary))

### Summary

**Unions in the Semiconductor Industry:**
- Unions are targeting semiconductor industry workers to organize them in line with President Biden's economic plans.
- Companies like Taiwan Semiconductor Manufacturing Co. and Intel Corp. stand to benefit from funding under the 2022 Chips Act, providing an opening for labor unions.
- The Communications Workers of America (CWA) is initiating efforts to organize chip-making workers, similar to the auto industry.

**Challenges and Opportunities:**
- Unionizing the chip sector faces obstacles due to the limited presence of unionized semiconductor workers in the US.
- While Intel has shown resistance to union discussions, the CWA has managed to secure labor peace negotiations with Micron, potentially setting the stage for future successful campaigns.

**Future Perspectives and Developments:**
- The Biden administration's push for a manufacturing boom driven by national security could utilize the Chips Act to promote organized labor.
- White House negotiations with advanced chip manufacturers could impact industry practices and labor relations.
- Scheduled political events, including speeches by Biden and Trump, the G7 summit, and the 2024 presidential debates, will continue to shape the landscape.

- Biden's nominations for the National Labor Relations Board (NLRB) aim to secure a Democratic majority until 2026, regardless of the election outcome.
- The nominations of Lauren McFerran and Joshua Ditelberg are contentious, drawing criticism from business advocates.
- Potential implications of Democratic control of the NLRB amidst a Republican administration raise concerns for the management community.

- Boeing collaborates with its labor union and the FAA to address safety concerns raised by employees through the Speak Up program.
- Efforts to enhance safety culture follow a recent incident involving a 737 Max aircraft, with Boeing outlining corrective actions in a forthcoming 90-day plan to the FAA.
- The company has witnessed increased employee submissions, provided safety training, and improved production processes.